# Fitting to the experimental data

In this notebook we will show how to load a CIF file, an experimental profile and how to perform a parameter fit.


#### Import Python packages

In [1]:
# esyScience, technique-independent
from easyCore import np
from easyCore.Fitting.Fitting import Fitter

# esyScience, diffraction
from easyDiffractionLib import Phases
from easyDiffractionLib.sample import Sample as Job
from easyDiffractionLib.interface import InterfaceFactory as Calculator
from easyDiffractionLib.Elements.Experiments.Pattern import Pattern1D
from easyDiffractionLib.Elements.Backgrounds.Point import PointBackground, BackgroundPoint
from easyDiffractionLib.Profiles.P1D import Instrument1DCWParameters

# Vizualization
import py3Dmol
import matplotlib.pyplot as plt

GSAS-II binary directory: /Users/asazonov/Library/Caches/pypoetry/virtualenvs/easydiffractionlib-x2vVD2Ys-py3.7/lib/python3.7/site-packages/GSASII/bindist
ImportError for wx/mpl in GSASIIctrlGUI: ignore if docs build


## --- Sample ---

#### Show a CIF file content

In [2]:
cif_fname = 'PbSO4.cif'

with open(cif_fname, 'r') as f:
    content = f.read()
    
print(content)

data_PbSO4

_space_group_name_H-M_alt   'P n m a'

_cell_length_a       8.480
_cell_length_b       5.398
_cell_length_c       6.958
_cell_angle_alpha   90.0
_cell_angle_beta    90.0
_cell_angle_gamma   90.0

loop_
 _atom_site_label
 _atom_site_type_symbol
 _atom_site_fract_x
 _atom_site_fract_y
 _atom_site_fract_z
 _atom_site_occupancy
 _atom_site_adp_type
 _atom_site_U_iso_or_equiv
  Pb  Pb   0.188   0.25   0.167   1.0   Uiso  0.01
  S   S    0.063   0.25   0.686   1.0   Uiso  0.01
  O1  O   -0.095   0.25   0.600   1.0   Uiso  0.01
  O2  O    0.181   0.25   0.543   1.0   Uiso  0.01
  O3  O    0.085   0.026  0.806   1.0   Uiso  0.01



#### Load structure from a CIF file

In [3]:
phases = Phases.from_cif_file(cif_fname)
phase = phases[0]

print(phases)
print(phase)

Collection of 1 phases.
Phase `PbSO4`


#### Visualise the structure

In [4]:
structure = py3Dmol.view()
structure.addModel(phase.to_cif_str(), 'cif')
structure.setStyle({'sphere':{'colorscheme':'Jmol','scale':.2},'stick':{'colorscheme':'Jmol','radius': 0.1}})
structure.addUnitCell()
structure.replicateUnitCell(2,2,1)
structure.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## --- Experiment ---

#### Show measured data as text

In [5]:
meas_fname = 'D1A@ILL.xye'

with open(meas_fname, 'r') as f:
    content = f.read()
    
print(content)

# PbSO4 D1A(ILL)(Rietveld Refinement Round Robin, R.J. Hill, JApC 25, 589 (1992)
       10.0000          220.0000         14.8324
       10.0500          214.0000         14.6287
       10.1000          219.0000         14.7986
       10.1500          224.0000         14.9666
       10.2000          198.0000         14.0712
       10.2500          229.0000         15.1327
       10.3000          224.0000         14.9666
       10.3500          216.0000         14.6969
       10.4000          202.0000         14.2127
       10.4500          229.0000         15.1327
       10.5000          202.0000         14.2127
       10.5500          215.0000         14.6629
       10.6000          215.0000         14.6629
       10.6500          196.0000         14.0000
       10.7000          235.0000         15.3297
       10.7500          207.0000         14.3875
       10.8000          205.0000         14.3178
       10.8500          238.0000         15.4272
       10.9000          202.0000     

#### Load the measured data

In [6]:
meas_x, meas_y, meas_e = np.loadtxt(meas_fname, unpack=True)

#### Visualize the measured data

In [7]:
%matplotlib widget
plt.plot(meas_x, meas_y, label='Imeas')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## --- Analysis ---

#### Create job with default parameters for the 1D powder neutron diffraction experiment with constant wavelength 

In [8]:
calculator = Calculator()

print(f"Current calculator engine: {calculator.current_interface_name}")

Current calculator engine: CrysPy


In [9]:
job = Job(phases=phases, parameters=Instrument1DCWParameters.default(), calculator=calculator)

Temp CIF: /var/folders/5q/6x3b8ryn5cn9hkg4lmlcpjyh0000gn/T/easydiffraction_temp.cif


#### Generate the calculated data

In [10]:
calc_y_cryspy = calculator.fit_func(meas_x)

#### Visualize both the measured and calculated data

In [11]:
%matplotlib widget
plt.plot(meas_x, meas_y, label='Imeas')
plt.plot(meas_x, calc_y_cryspy, label='Icalc (CrysPy)')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Set scale manually

In [12]:
job.pattern.scale = 100

In [13]:
calc_y_cryspy = calculator.fit_func(meas_x)

%matplotlib widget
plt.plot(meas_x, meas_y, label='Imeas')
plt.plot(meas_x, calc_y_cryspy, label='Icalc (CrysPy)')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Set wavelength manually

In [14]:
job.parameters.wavelength = 1.912

In [15]:
calc_y_cryspy = calculator.fit_func(meas_x)

%matplotlib widget
plt.plot(meas_x, meas_y, label='Imeas')
plt.plot(meas_x, calc_y_cryspy, label='Icalc (CrysPy)')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Set background points manually

In [16]:
bkg = PointBackground(linked_experiment='PbSO4')

bkg.append(BackgroundPoint.from_pars(meas_x[0], 200))
bkg.append(BackgroundPoint.from_pars(meas_x[-1], 250))

job.set_background(bkg)

In [17]:
calc_y_cryspy = calculator.fit_func(meas_x)

%matplotlib widget
plt.plot(meas_x, meas_y, label='Imeas')
plt.plot(meas_x, calc_y_cryspy, label='Icalc (CrysPy)')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Define parameters to optimize

In [18]:
job.pattern.scale.fixed = False
job.pattern.zero_shift.fixed = False
job.parameters.resolution_u.fixed = False
job.parameters.resolution_v.fixed = False
job.parameters.resolution_w.fixed = False
job.backgrounds[0][0].y.fixed = False
job.backgrounds[0][1].y.fixed = False

In [19]:
print(job.pattern.scale)
print(job.pattern.zero_shift)
print(job.parameters.resolution_u)
print(job.parameters.resolution_v)
print(job.parameters.resolution_w)
print(job.backgrounds[0][0])
print(job.backgrounds[0][1])

<Parameter 'scale': 100.0+/-0, bounds=[-inf:inf]>
<Parameter 'zero_shift': 0.0+/-0, bounds=[-inf:inf]>
<Parameter 'resolution_u': 0.0002+/-0, bounds=[-inf:inf]>
<Parameter 'resolution_v': -0.0002+/-0, bounds=[-inf:inf]>
<Parameter 'resolution_w': 0.012+/-0, bounds=[-inf:inf]>
<BackgroundPoint '10,0_deg': 200.0+/-0, bounds=[-inf:inf]>
<BackgroundPoint '120,0_deg': 250.0+/-0, bounds=[-inf:inf]>


#### Initalize the fitting engine and perform the fit

In [20]:
fitter = Fitter(job, calculator.fit_func)

print(f"Available minimizers: {fitter.available_engines}")
print(f"Current minimizer: {fitter.current_engine.name}")
print(f"Available methods of current minimizers: {fitter.available_methods()}")

Available minimizers: ['lmfit', 'bumps']
Current minimizer: lmfit
Available methods of current minimizers: ['leastsq', 'least_squares', 'differential_evolution', 'basinhopping', 'ampgo', 'nelder', 'lbfgsb', 'powell', 'cg', 'newton', 'cobyla', 'bfgs']


In [21]:
result = fitter.fit(meas_x, meas_y, weights=1/meas_e, method='least_squares')

print("The fit has been successful: {}".format(result.success))
if result.success:    
    print("The gooodness of fit (chi2) is: {}".format(result.reduced_chi))
    print(job.pattern.scale)
    print(job.pattern.zero_shift)
    print(job.parameters.resolution_u)
    print(job.parameters.resolution_v)
    print(job.parameters.resolution_w)
    print(job.backgrounds[0][0])
    print(job.backgrounds[0][1])

The fit has been successful: True
The gooodness of fit (chi2) is: 31.615632345412287
<Parameter 'scale': 600.2020579794987+/-0, bounds=[-inf:inf]>
<Parameter 'zero_shift': 0.1201907917138295+/-0, bounds=[-inf:inf]>
<Parameter 'resolution_u': 0.18432993308244003+/-0, bounds=[-inf:inf]>
<Parameter 'resolution_v': -0.4530089279130012+/-0, bounds=[-inf:inf]>
<Parameter 'resolution_w': 0.46678147385907176+/-0, bounds=[-inf:inf]>
<BackgroundPoint '10,0_deg': 198.88539922987408+/-0, bounds=[-inf:inf]>
<BackgroundPoint '120,0_deg': 239.51362568654585+/-0, bounds=[-inf:inf]>


In [22]:
calc_y_cryspy = calculator.fit_func(meas_x)

%matplotlib widget
plt.plot(meas_x, meas_y, label='Imeas')
plt.plot(meas_x, calc_y_cryspy, label='Icalc (CrysPy)')
plt.plot(meas_x, meas_y-calc_y_cryspy, label='Imeas-Icalc (CrysPy)')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Change calculator engine to CrysFML

In [23]:
job.interface.switch('CrysFML')
job.update_bindings()

print(f"Current calculator engine: {job.interface.current_interface_name}")
print(f"Current minimizer: {fitter.current_engine.name}")

Current calculator engine: CrysFML
Current minimizer: lmfit


#### Show results of both CrysPy and CrysFML calculations (before fitting)

In [24]:
calc_y_crysfml = calculator.fit_func(meas_x)

%matplotlib widget
plt.plot(meas_x, meas_y, label='Imeas')
plt.plot(meas_x, calc_y_cryspy, label='Icalc (CrysPy)')
plt.plot(meas_x, calc_y_crysfml, label='Icalc (CrysFML)')
plt.legend()





+ calculate A: 0.0011 s
+ calculate B: 0.0013 s
+ calculate C: 0.0001 s
+ reflection_list = CFML_api.ReflectionList: 0.0001 s
+ reflection_list.compute_structure_factors: 0.0003 s
+ set reflection_list: 0.0064 s
+ diffraction_pattern = CFML_api.DiffractionPattern: 0.0025 s
+ calculate D: 0.0019 s
+ calculate E: 0.0039 s
+ calculate F: 0.0000 s


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Perform the fit with CrysFML

In [25]:
result = fitter.fit(meas_x, meas_y, weights=1/meas_e, method='least_squares')

print("The fit has been successful: {}".format(result.success))
if result.success:    
    print("The gooodness of fit (chi2) is: {}".format(result.reduced_chi))
    print(job.pattern.scale)
    print(job.pattern.zero_shift)
    print(job.parameters.resolution_u)
    print(job.parameters.resolution_v)
    print(job.parameters.resolution_w)
    print(job.backgrounds[0][0])
    print(job.backgrounds[0][1])

FitError: Unable to return value:
'NoneType' object has no attribute 'strip'
Something has gone wrong with the fit

#### Show results of both CrysPy and CrysFML calculations (after fitting)

In [ ]:
calc_y_crysfml = calculator.fit_func(meas_x)

%matplotlib widget
plt.plot(meas_x, meas_y, label='Imeas')
plt.plot(meas_x, calc_y_cryspy, label='Icalc (CrysPy)')
plt.plot(meas_x, calc_y_crysfml, label='Icalc (CrysFML)')
plt.legend()